# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('customer_product_sales.csv')

In [4]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


In [6]:
data.columns

Index(['CustomerID', 'FirstName', 'LastName', 'SalesID', 'ProductID',
       'ProductName', 'Quantity'],
      dtype='object')

## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [7]:
data_grouped = data.groupby(['CustomerID', 'ProductName']).agg({'Quantity': 'sum'}).reset_index()

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [8]:
product_by_customer = pd.pivot_table(data, values='Quantity', index='ProductName', columns='CustomerID', aggfunc='sum').fillna(0)

## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [9]:
from scipy.spatial.distance import pdist, squareform

customer_by_product = pd.pivot_table(data, values='Quantity', index='CustomerID', columns='ProductName', aggfunc='sum').fillna(0)

customer_similarity = 1 / (1 + squareform(pdist(customer_by_product, metric='euclidean')))

## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [14]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


In [15]:
import numpy as np

customer_id = 61288 
customer_index = customer_by_product.index.get_loc(customer_id)

customer_similarity_values = customer_similarity[customer_index]

most_similar_indices = np.argsort(customer_similarity_values)[::-1][1:6]

most_similar_customers = customer_by_product.iloc[most_similar_indices].index.tolist()

print("Top 5 most similar customers for CustomerID", customer_id, ":", most_similar_customers)

Top 5 most similar customers for CustomerID 61288 : [15166, 5968, 8539, 5224, 5104]


## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [17]:
ids = [15166, 5968, 8539, 5224, 5104]
selected_records = data[data['CustomerID'].isin(ids)]
selected_records

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
25,15166,Deanna,Hogan,347467,229,Bread - Hot Dog Buns,4
326,15166,Deanna,Hogan,4442792,153,Arizona - Green Tea,4
449,5968,Trevor,Gilbert,954458,432,Beer - Original Organic Lager,2
533,5224,Paige,Gay,3387595,432,Beer - Original Organic Lager,2
535,5104,Camille,Lozano,840942,432,Beer - Original Organic Lager,2
...,...,...,...,...,...,...,...
68323,8539,Jon,Giles,4498054,178,Pernod,3
68332,15166,Deanna,Hogan,4317094,178,Pernod,4
68462,8539,Jon,Giles,1077283,211,Wine - Prosecco Valdobiaddene,3
68463,8539,Jon,Giles,6141083,211,Wine - Prosecco Valdobiaddene,3


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [18]:
product_totals = selected_records.groupby('ProductName')['Quantity'].agg('sum').reset_index()
ranked_products = product_totals.sort_values(by='Quantity', ascending=False)

print(ranked_products)

                    ProductName  Quantity
19           Beef - Rib Eye Aaa        12
100                Jagermeister        12
54          Chicken - Soup Base        12
113        Lime Cordial - Roses        10
108        Lamb - Pieces, Diced        10
..                          ...       ...
161            Salsify, Organic         2
160   Salmon Steak - Cohoe 8 Oz         2
159        Salmon - Sockeye Raw         2
158  Salmon - Atlantic, Skin On         2
46           Cheese - Camembert         2

[231 rows x 2 columns]


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [19]:
product_totals = selected_records.groupby('ProductName')['Quantity'].agg('sum').reset_index()
ranked_products = product_totals.sort_values(by='Quantity', ascending=False)

print(ranked_products)

                    ProductName  Quantity
19           Beef - Rib Eye Aaa        12
100                Jagermeister        12
54          Chicken - Soup Base        12
113        Lime Cordial - Roses        10
108        Lamb - Pieces, Diced        10
..                          ...       ...
161            Salsify, Organic         2
160   Salmon Steak - Cohoe 8 Oz         2
159        Salmon - Sockeye Raw         2
158  Salmon - Atlantic, Skin On         2
46           Cheese - Camembert         2

[231 rows x 2 columns]


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.